In [1]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.4 MB/s eta 0:00:0000:01


In [2]:
import torch
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import GCNConv, GATConv, GINConv
from torch_geometric.datasets import CoraFull, DeezerEurope
from torch_geometric.utils import train_test_split_edges, negative_sampling, subgraph
from torch_geometric.utils import to_dense_adj, to_undirected, train_test_split_edges
from sklearn.model_selection import train_test_split

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

corafull = CoraFull(root='CoraFull')[0]
deezerEurope = DeezerEurope(root='DeezerEurope')[0]

datasets = {'CoraFull': corafull.to(device), 'DeezerEurope': deezerEurope.to(device)}

Processing...
Done!
Processing...
Done!


In [3]:
print (corafull)
print (deezerEurope)

Data(x=[19793, 8710], edge_index=[2, 126842], y=[19793])
Data(x=[28281, 128], edge_index=[2, 185504], y=[28281])


In [4]:
from sklearn.metrics import pairwise_distances
import numpy as np

def compute_nodes_in_triangle(edge_index, num_nodes):
    """
    Identify nodes in Q: nodes part of at least one triangle in the graph.
    """
    adj = to_dense_adj(edge_index, max_num_nodes=num_nodes).squeeze(0)
    triangles = torch.mm(adj, adj.T) * adj  # Triangle adjacency
    Q = torch.nonzero(torch.sum(triangles, dim=1)).squeeze()
    return Q[:1000]

In [5]:
# Split Datasets into train, test and validation
train_ratio, val_ratio, test_ratio = 0.6, 0.2, 0.2

num_nodes = {}
edge_index = {}

train_nodes = {}
train_data = {}
train_edge_index = {}

test_nodes = {}
test_data = {}
test_edge_index = {}
test_neg_edge_index = {}
Q = {}

val_nodes = {}
val_data = {}
val_edge_index = {}
val_neg_edge_index = {}

for dataset_name in datasets:
    num_nodes[dataset_name] = datasets[dataset_name].num_nodes
    edge_index[dataset_name] = datasets[dataset_name].edge_index

    edge_splits = train_test_split_edges(datasets[dataset_name], test_ratio=test_ratio, val_ratio=val_ratio)
    
    train_edge_index[dataset_name] = to_undirected(edge_splits.train_pos_edge_index)
    test_edge_index[dataset_name] = to_undirected(edge_splits.test_pos_edge_index)
    val_edge_index[dataset_name] = to_undirected(edge_splits.val_pos_edge_index)

    train_nodes[dataset_name] = torch.sort(torch.unique(train_edge_index[dataset_name]))[0].to(device)
    train_data[dataset_name] = datasets[dataset_name].x[train_nodes[dataset_name]].to(device)
    train_edge_index[dataset_name] = subgraph(train_nodes[dataset_name], train_edge_index[dataset_name], relabel_nodes=True)[0].to(device)

    
    
    test_nodes[dataset_name] = torch.sort(torch.unique(test_edge_index[dataset_name]))[0].to(device)
    test_data[dataset_name] = datasets[dataset_name].x[test_nodes[dataset_name]].to(device)
    test_edge_index[dataset_name] = subgraph(test_nodes[dataset_name], test_edge_index[dataset_name], relabel_nodes=True)[0].to(device)

    test_neg_edge_index[dataset_name] = negative_sampling(edge_index=test_edge_index[dataset_name],
                                                          num_nodes=test_nodes[dataset_name].shape[0],
                                                          num_neg_samples=test_edge_index[dataset_name].shape[1]).to(device)

    Q[dataset_name] = compute_nodes_in_triangle(test_edge_index[dataset_name], test_nodes[dataset_name].shape[0])
    
    
    val_nodes[dataset_name] = torch.sort(torch.unique(val_edge_index[dataset_name]))[0].to(device)
    val_data[dataset_name] = datasets[dataset_name].x[val_nodes[dataset_name]].to(device)
    val_edge_index[dataset_name] = subgraph(val_nodes[dataset_name], val_edge_index[dataset_name], relabel_nodes=True)[0].to(device)

    val_neg_edge_index[dataset_name] = negative_sampling(edge_index=val_edge_index[dataset_name], 
                                                         num_nodes=val_nodes[dataset_name].shape[0],
                                                         num_neg_samples=val_edge_index[dataset_name].shape[1]).to(device)

/opt/conda/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [6]:
print (train_edge_index['CoraFull'].shape)
print (test_edge_index['CoraFull'].shape)
print (val_edge_index['CoraFull'].shape)

print (train_edge_index['DeezerEurope'].shape)
print (test_edge_index['DeezerEurope'].shape)
print (val_edge_index['DeezerEurope'].shape)

torch.Size([2, 76106])
torch.Size([2, 25368])
torch.Size([2, 25368])
torch.Size([2, 111304])
torch.Size([2, 37100])
torch.Size([2, 37100])


In [7]:
# GNN Model Definition
class GNNModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels=64, out_channels=64, gnn_type="GCN"):
        super(GNNModel, self).__init__()
        if gnn_type == "GCN":
            self.conv1 = GCNConv(in_channels, hidden_channels)
            self.conv2 = GCNConv(hidden_channels, out_channels)
        elif gnn_type == "GAT":
            self.conv1 = GATConv(in_channels, hidden_channels)
            self.conv2 = GATConv(hidden_channels, out_channels)
        elif gnn_type == "GIN":
            self.conv1 = GINConv(torch.nn.Sequential(
                torch.nn.Linear(in_channels, hidden_channels),
                torch.nn.ReLU(),
                torch.nn.Linear(hidden_channels, hidden_channels)
            ))
            self.conv2 = GINConv(torch.nn.Sequential(
                torch.nn.Linear(hidden_channels, out_channels),
                torch.nn.ReLU(),
                torch.nn.Linear(out_channels, out_channels)
            ))
        else:
            raise ValueError("Unsupported GNN type")

    def forward(self, input, edge_index):
        hidden_output = F.relu(self.conv1(input, edge_index))
        output = self.conv2(hidden_output, edge_index)
        return output

In [8]:
def auc_loss(node_embeddings, pos_edge_index, neg_edge_index, margin=1.0):
    # Positive edge scores
    pos_u, pos_v = pos_edge_index
    pos_scores = F.cosine_similarity(node_embeddings[pos_u], node_embeddings[pos_v])

    # Negative edge scores
    neg_u, neg_v = neg_edge_index
    neg_scores = F.cosine_similarity(node_embeddings[neg_u], node_embeddings[neg_v])
    
    # Margin-based ranking loss
    loss = F.relu(margin + neg_scores - pos_scores).mean()
    return loss

In [9]:
from rich.console import Console
from rich.table import Table

console = Console()

def train(model, optimizer, train_data, train_edge_index, val_data, val_edge_index, val_neg_edge_index, epochs=200, val_interval=20):
    model.to(device)
    model.train()

    for epoch in range(1, epochs + 1):
        train_neg_edge_index = negative_sampling(edge_index=train_edge_index, 
                                                 num_nodes=train_data.shape[0], 
                                                 num_neg_samples=train_edge_index.shape[1]).to(device)

        node_embeddings = model(train_data, train_edge_index)
        loss = auc_loss(node_embeddings, train_edge_index, train_neg_edge_index)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if epoch % val_interval == 0:
            model.eval()
            with torch.no_grad():
                node_embeddings = model(val_data, val_edge_index)
                val_loss = auc_loss(node_embeddings, val_edge_index, val_neg_edge_index)
            model.train()

            console.print(f"Epoch {epoch:3d} | [cyan]Train Loss:[/] {loss:.5f} | [green]Val Loss:[/] {val_loss:.5f}")

In [10]:
models = {}

for dataset_name in datasets:
    models[dataset_name] = {'GCN': None, 'GAT': None, 'GIN': None}
    
    for gnn_model in models[dataset_name]:
        console.rule(f"[bold blue]Dataset: {dataset_name}, GNN Model: {gnn_model}")
        
        model = GNNModel(datasets[dataset_name].num_features, gnn_type=gnn_model)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
    
        train(model, optimizer, train_data[dataset_name], train_edge_index[dataset_name], 
              val_data[dataset_name], val_edge_index[dataset_name], val_neg_edge_index[dataset_name])

        models[dataset_name][gnn_model] = model

──────────────────────────────────────── Dataset: CoraFull, GNN Model: GCN ────────────────────────────────────────

Epoch  20 | Train Loss: 0.29385 | Val Loss: 0.27606

Epoch  40 | Train Loss: 0.19820 | Val Loss: 0.18660

Epoch  60 | Train Loss: 0.17116 | Val Loss: 0.16203

Epoch  80 | Train Loss: 0.15281 | Val Loss: 0.14490

...

Epoch 120 | Train Loss: 0.22992 | Val Loss: 0.22714

Epoch 140 | Train Loss: 0.22621 | Val Loss: 0.22343

Epoch 160 | Train Loss: 0.22306 | Val Loss: 0.22472

Epoch 180 | Train Loss: 0.22172 | Val Loss: 0.22734

Epoch 200 | Train Loss: 0.21770 | Val Loss: 0.22414

In [11]:
import numpy as np

def get_evaluation(scores, labels, K_values):
    ranked_nodes = torch.argsort(scores, descending=True)
    rankings = torch.argsort(ranked_nodes)

    precision = {}
    for k in K_values:
        precision[k] = (labels[ranked_nodes[:k]].sum() / k).item()
    rank_of_first_positive = torch.min(rankings[torch.nonzero(labels)]).item() + 1
    mrr = 1 / rank_of_first_positive

    return precision, mrr

In [12]:
import numpy as np

def evaluate_inference(model, test_data, test_edge_index, Q, K_values):
    model.eval()
    with torch.no_grad():
        precision_scores = {k: [] for k in K_values}
        mrr_scores = []
        adj = to_dense_adj(test_edge_index, max_num_nodes=test_data.shape[0]).squeeze(0)
        degree = torch.sum(adj, dim=1)
        
        embeddings = model(test_data, test_edge_index)
        
        for q in Q:
            scores = F.cosine_similarity(embeddings[q].unsqueeze(0), embeddings, dim=1)
            scores[q] = float('-inf')
            labels = adj[q]
    
            p, m = get_evaluation(scores, labels, K_values)
            for k in K_values:
                precision_scores[k].append(p[k])
            mrr_scores.append(m)
        
        # Average metrics
        precision_avg = {k: np.mean(precision_scores[k]) for k in K_values}
        mrr_avg = np.mean(mrr_scores)
        
    return precision_avg, mrr_avg

In [13]:
def adamic_adar(test_data, test_edge_index, Q, K_values):
    precision_scores = {k: [] for k in K_values}
    mrr_scores = []
    adj = to_dense_adj(test_edge_index, max_num_nodes=test_data.shape[0]).squeeze(0).to(device)
    degree = torch.sum(adj, dim=1).to(device)
    
    for q in Q:
        scores = adj[q] * adj
        scores = (1 / torch.log(degree.unsqueeze(0) * (scores > 0))).sum(dim=1)
        # scores = torch.stack([(1 / torch.log(degree[row > 0])).sum() for row in scores])
        scores[q] = float('-inf')
        labels = adj[q]
        
        p, m = get_evaluation(scores, labels, K_values)
        for k in K_values:
            precision_scores[k].append(p[k])
        mrr_scores.append(m)
    
    # Average metrics
    precision_avg = {k: np.mean(precision_scores[k]) for k in K_values}
    mrr_avg = np.mean(mrr_scores)
    
    return precision_avg, mrr_avg

def common_neighbors(test_data, test_edge_index, Q, K_values):
    precision_scores = {k: [] for k in K_values}
    mrr_scores = []
    adj = to_dense_adj(test_edge_index, max_num_nodes=test_data.shape[0]).squeeze(0).to(device)
    degree = torch.sum(adj, dim=1).to(device)

    for q in Q:
        scores = torch.sum(adj[q] * adj, dim=1)
        scores[q] = float('-inf')
        labels = adj[q]

        p, m = get_evaluation(scores, labels, K_values)
        for k in K_values:
            precision_scores[k].append(p[k])
        mrr_scores.append(m)
    
    # Average metrics
    precision_avg = {k: np.mean(precision_scores[k]) for k in K_values}
    mrr_avg = np.mean(mrr_scores)
    
    return precision_avg, mrr_avg

In [16]:
import time

K_values = [1, 5, 10]

for dataset_name in datasets:
    console.rule(f"[bold blue]Dataset: {dataset_name}")

    table = Table(show_header=True, header_style="bold green")
    table.add_column("GNN Models", style="dim", justify="center")
    for k in K_values:
        table.add_column(f"Precision@{k}", style="bold magenta", justify="center")
    table.add_column("MRR", style="bold magenta", justify="center")
    table.add_column("Inference Time (s)", style="bold red", justify="center")

    for gnn_model in models[dataset_name]:
        start_time = time.time()
        model = models[dataset_name][gnn_model]
        precision_avg, mrr_avg = evaluate_inference(model, test_data[dataset_name], test_edge_index[dataset_name], Q[dataset_name], K_values)
        end_time = time.time()
        
        row = [gnn_model]
        for k in K_values:
            row.append(f"{precision_avg[k]:.5f}")
        row.append(f"{mrr_avg:.5f}")
        row.append(f"{(end_time - start_time):.5f}")
        table.add_row(*row)

    
    # Adamic Adar
    start_time = time.time()
    precision_avg, mrr_avg = adamic_adar(test_data[dataset_name], test_edge_index[dataset_name], Q[dataset_name], K_values)
    end_time = time.time()
    
    row = ["Adamic-Adar"]
    for k in K_values:
        row.append(f"{precision_avg[k]:.5f}")
    row.append(f"{mrr_avg:.5f}")
    row.append(f"{(end_time - start_time):.5f}")
    table.add_row(*row)
    
    # Common Neighbors
    start_time = time.time()
    precision_avg, mrr_avg = common_neighbors(test_data[dataset_name], test_edge_index[dataset_name], Q[dataset_name], K_values)
    end_time = time.time()
    
    row = ["Common-Neighbors"]
    for k in K_values:
        row.append(f"{precision_avg[k]:.5f}")
    row.append(f"{mrr_avg:.5f}")
    row.append(f"{(end_time - start_time):.5f}")
    table.add_row(*row)
    
    console.print(table)

──────────────────────────────────────────────── Dataset: CoraFull ────────────────────────────────────────────────

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃    GNN Models    ┃ Precision@1 ┃ Precision@5 ┃ Precision@10 ┃   MRR   ┃ Inference Time (s) ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│       GCN        │   0.72771   │   0.42217   │   0.27687    │ 0.81292 │      0.68034       │
│       GAT        │   0.70723   │   0.41831   │   0.27277    │ 0.79579 │      0.57193       │
│       GIN        │   0.38795   │   0.23711   │   0.16783    │ 0.51909 │      0.59798       │
│   Adamic-Adar    │   0.33133   │   0.23398   │   0.17024    │ 0.53454 │      22.48552      │
│ Common-Neighbors │   0.18916   │   0.19084   │   0.14807    │ 0.38438 │      5.86862       │
└──────────────────┴─────────────┴─────────────┴──────────────┴─────────┴────────────────────┘

────────────────────────────────────────────── Dataset: DeezerEurope ──────────────────────────────────────────────

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃    GNN Models    ┃ Precision@1 ┃ Precision@5 ┃ Precision@10 ┃   MRR   ┃ Inference Time (s) ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│       GCN        │   0.71399   │   0.38149   │   0.23491    │ 0.78927 │      0.70380       │
│       GAT        │   0.78864   │   0.43764   │   0.26761    │ 0.85603 │      0.70083       │
│       GIN        │   0.40589   │   0.22208   │   0.15216    │ 0.53081 │      0.69888       │
│   Adamic-Adar    │   0.31651   │   0.21052   │   0.15279    │ 0.49881 │      50.55869      │
│ Common-Neighbors │   0.22713   │   0.18002   │   0.13985    │ 0.40266 │      12.79649      │
└──────────────────┴─────────────┴─────────────┴──────────────┴─────────┴────────────────────┘